In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import cv2

from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout
from keras.models import Model
from keras.models import Sequential

from sklearn.neighbors import NearestNeighbors

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
vgg16 = keras.applications.VGG16(weights='imagenet', include_top=True, pooling='max', input_shape=(224, 224, 3))
basemodel = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
basemodel.summary()

553467096/553467096 [==============================] - 17s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

In [4]:
def get_feature_vector(image_path, file_list_index=''):
    img = mpimg.imread(image_path)
    if len(img.shape) ==3:
        img1 = cv2.resize(img, (224, 224))
        feature_vector = basemodel.predict(img1.reshape(1, 224, 224, 3))
        return feature_vector
    print(f'{file_list_index} {os.path.basename(image_path)} has only black and white channels. Not supported')
    return None

In [5]:
folder_path = '/kaggle/input/top100actors-top100actresses' # provide your path here

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
for n, i in enumerate(file_list): print(n, os.path.splitext(os.path.basename(i))[0])

0 clark_gable
1 elizabeth_banks
2 frances_mcdormand
3 daisy_ridley
4 rachel_mcadams
5 orson_welles
6 saoirse_ronan
7 jeremy_irons
8 julia_roberts
9 anthony_hopkins
10 jon_voight
11 natalie_portman
12 jennifer_lawrence
13 cate_blanchett
14 hilary_swank
15 alicia_vikander
16 jack_lemmon
17 liv_ullmann
18 anna_kendrick
19 will_smith
20 james_stewart
21 philip_seymour_hoffman
22 daniel_day-lewis
23 heath_ledger
24 robert_downey_jr.
25 grace_kelly
26 alyson_hannigan
27 sigourney_weaver
28 kim_basinger
29 richard_burton
30 john_wayne
31 marion_cotillard
32 anthony_quinn
33 carey_mulligan
34 michelle_pfeiffer
35 leonardo_dicaprio
36 kirsten_dunst
37 christopher_walken
38 sally_hawkins
39 julianne_moore
40 don_cheadle
41 ben_kingsley
42 demi_moore
43 charles_chaplin
44 talia_shire
45 tom_hanks
46 alan_arkin
47 elizabeth_olsen
48 blake_lively
49 brad_pitt
50 jennifer_jason_leigh
51 colin_firth
52 kirk_douglas
53 brie_larson
54 carrie_fisher
55 kate_mara
56 jack_nicholson
57 tommy_lee_jones
58 c

In [6]:
vectors = np.array([get_feature_vector(image, file_list_index=n) for n, image in enumerate(file_list)])

0 clark_gable.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 332ms/step
16 jack_lemmon.jpg has only black and white channels. Not supported
17 liv_ullmann.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 334ms/step
28 kim_basinger.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 330ms/step
32 anthony_quinn.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 330ms/step
43 charles_chaplin.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 384ms/step
52 kirk_douglas.jpg has only black and white channels. Not supported
1/1 [==============================] - 0s 335ms/step
62 jason_robards.jpg has only black and white channels. Not supported
63 edward_g._robinson.jpg has only black and white channels. Not supported
64 ingrid_bergman.jpg has only black and white channel

/tmp/ipykernel_32/1075150370.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectors = np.array([get_feature_vector(image, file_list_index=n) for n, image in enumerate(file_list)])


In [7]:
#def similarity_check(id1, id2, file_list, vectors):
#    print(os.path.splitext(os.path.basename(file_list[id1]))[0],'vs', os.path.splitext(os.path.basename(file_list[id2]))[0])
#    person1 = (os.path.splitext(os.path.basename(file_list[id1]))[0], vectors[id1])
#    person2 = (os.path.splitext(os.path.basename(file_list[id2]))[0], vectors[id2])
#    if type(vectors[id1]) and type(vectors[id2]) is not type(None): 
#        cos_sim = calculate_similarity(person1[1][0], person2[1][0])
#        euc_dist = np.linalg.norm(person1[1][0] - person2[1][0])
#        print('cosine based similarity', round(cos_sim,4) ) # the bigger the better
#        print('eucleadean distance',euc_dist) # the lower the better
#        return (person1[0], person2[0], cos_sim, euc_dist)
#    print('id for black and white photo detected. Not possible to check similarity')
#   return (person1[0], person2[0], 'Failed to compare', 'Failed to compare')

In [21]:
vectors_no_none = np.array([i[0] for i in vectors if type(i)!=type(None)])

df = pd.DataFrame([os.path.splitext(os.path.basename(i))[0] for i in file_list])\
        .rename(columns={0:'name'})
df = pd.concat([df,pd.DataFrame(vectors)], axis=1).rename(columns={0:'vector'})

vectors_df = df[~df['vector'].isna()].reset_index(names='old_index')
vectors_df.head()

In [20]:
knn = NearestNeighbors(n_neighbors=len(vectors_no_none), n_jobs=-1)

knn.fit(vectors_no_none)

distances, indices = knn.kneighbors(vectors_no_none)

array([[  0,  86,  46, ...,  71,  43, 137],
       [  1, 101, 105, ...,  63,  43, 137],
       [  2,  79,  38, ...,  63,  43, 137],
       ...,
       [166,  84,  60, ...,  63, 137,  48],
       [167,  13,  68, ...,  92,  43, 137],
       [168, 157, 159, ...,  43,  63, 137]])

In [45]:
df_with_neighbors = pd.concat([vectors_df, pd.DataFrame(indices[:,1:])], axis=1)
df_with_neighbors[df_with_neighbors.columns[3:]] = df_with_neighbors[df_with_neighbors.columns[3:]].applymap(lambda i: df_with_neighbors.name.iloc[i])
df_with_neighbors.head()

,old_index,name,vector,0,1,2,3,4,5,6,...,158,159,160,161,162,163,164,165,166,167
0,1,elizabeth_banks,"[[0.0, 0.0, 0.0, 0.0, 1.7654319, 0.0, 0.0, 0.0...",lauren_cohan,brie_larson,chlo�_grace_moretz,joan_allen,heath_ledger,hilary_swank,jennifer_jason_leigh,...,javier_bardem,christoph_waltz,benicio_del_toro,jamie_foxx,matthew_mcconaughey,kate_mara,matt_damon,hugh_jackman,brad_pitt,russell_crowe
1,2,frances_mcdormand,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.858412, 0.0, 0.0,...",katherine_parkinson,sean_penn,robert_redford,tilda_swinton,joaquin_phoenix,heath_ledger,rosamund_pike,...,robert_mitchum,kirsten_dunst,jamie_foxx,jeremy_irons,kate_mara,jennifer_aniston,jennifer_connelly,vera_farmiga,brad_pitt,russell_crowe
2,3,daisy_ridley,"[[0.0, 0.0, 0.0, 0.89682543, 0.0, 0.0, 0.0, 0....",evangeline_lilly,talia_shire,keira_knightley,rosamund_pike,charlize_theron,kate_winslet,blake_lively,...,philip_seymour_hoffman,janet_leigh,hugh_jackman,christoph_waltz,jamie_foxx,matt_damon,robert_mitchum,vera_farmiga,brad_pitt,russell_crowe
3,4,rachel_mcadams,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.50915027, 0.0, 0....",scarlett_johansson,m�lanie_laurent,emma_stone,chlo�_grace_moretz,l�a_seydoux,brie_larson,julie_delpy,...,kate_mara,javier_bardem,peter_o__toole,christoph_waltz,hugh_jackman,matt_damon,benicio_del_toro,jamie_foxx,brad_pitt,russell_crowe
4,5,orson_welles,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8189506, 0.0...",joaquin_phoenix,james_stewart,alan_rickman,charlton_heston,jim_carrey,woody_allen,katherine_parkinson,...,octavia_spencer,brad_pitt,katheryn_winnick,janet_leigh,kirsten_dunst,jennifer_aniston,grace_kelly,vera_farmiga,russell_crowe,kate_mara


In [46]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=3, random_state=0, perplexity=12)
projections = tsne.fit_transform(vectors_no_none, )

plotting_df = pd.concat([vectors_df, pd.DataFrame(projections)], axis=1)

fig = px.scatter_3d(
    plotting_df, x=0, y=1, z=2, hover_data='name'
)
fig.update_traces(marker_size=4)
fig.show()